In [1]:
!pip install transformers
!pip install sentencepiece
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 2.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
import torch
from transformers import pipeline
import os


In [3]:
# load model
dolly_pipeline = pipeline(model="databricks/dolly-v2-3b",
                            torch_dtype=torch.bfloat16,
                            trust_remote_code=True,
                            device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/819 [00:00<?, ?B/s]

instruct_pipeline.py:   0%|          | 0.00/9.16k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/databricks/dolly-v2-3b:
- instruct_pipeline.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/5.68G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/450 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
def get_completion_dolly(input):
    # Step 1: Generate Executive Summary
    summary_prompt = f"Please summarize the following project description into an executive summary:\n\n{input}\n\nExecutive Summary:"
    summary_response = dolly_pipeline(summary_prompt, max_new_tokens=100)
    executive_summary = summary_response[0]["generated_text"]

    # Step 2: Classify into Pillars
    classification_system = f"""
    You are an expert in classifying projects into specific pillars.
    Here are the three main pillars of TechNova Solutions:

    1. **Innovation and Creativity**
    - Characteristics: Cutting-edge technology, experimentation, new ideas, unique solutions, R&D, AI, quantum computing, IoT, blockchain.
    - Keywords: innovation, creativity, R&D, AI, quantum, IoT, blockchain.

    2. **Customer-Centric Approach**
    - Characteristics: Customer satisfaction, personalized solutions, customer feedback, support systems, CRM, analytics.
    - Keywords: customer, personalized, support, CRM, feedback, satisfaction.

    3. **Sustainability and Social Responsibility**
    - Characteristics: Environmental impact, sustainability, eco-friendly, community engagement, renewable energy, ethical practices.
    - Keywords: sustainability, eco-friendly, community, renewable energy, ethical, social responsibility.

    Given the executive summary of a project, classify it into one of the above pillars.
    """
    classification_prompt = f"#### System: {classification_system}\n#### User: \n{executive_summary}\n\n#### Response from Dolly-v2-3b:"
    classification_response = dolly_pipeline(classification_prompt, max_new_tokens=100)
    classification = classification_response[0]["generated_text"]

    return executive_summary, classification



In [9]:

project_description = "OptiTrac Ocular Surveillance System: The project entails the development of a sophisticated software infrastructure aimed at augmenting employee productivity surveillance at XYZ Tech. Central to this system is the implementation of ocular presence detection via advanced oculographic tracking technologies. This process encompasses the utilization of high-fidelity computer vision algorithms coupled with deep learning paradigms, specifically leveraging convolutional neural networks (CNNs), to scrutinize real-time webcam data streams. The objective is to ascertain the presence and attentiveness of employees by analyzing minute ocular and facial micro-movements, thereby determining engagement levels during operational hours. The integration of these methodologies necessitates extensive training on voluminous annotated datasets to enhance accuracy and mitigate erroneous detections, ensuring a robust and seamless monitoring mechanism."

# Get the executive summary and classification
executive_summary, classification = get_completion_dolly(project_description)

print("Executive Summary:", executive_summary)
print("Classification:", classification)

Executive Summary: The OptiTrac Ocular Surveillance System will automate surveillance of employee productivity via advanced computer vision, deep learning and annotated datasets. This is part of a larger project that also entails development of ocular tracking technology and associated infrastructure. This technology will be utilized in conjunction with the automated surveillance to monitor employee attentiveness and productivity levels.
Classification: Innovation and Creativity
